In [8]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

# Получаем ссылки на каждую букву алфавита

In [ ]:
page_link = 'https://www.azlyrics.com/'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
html = response.content
soup = BeautifulSoup(html,'lxml')
objs = soup.findAll('a', attrs = {'class':'btn btn-menu'})
links=[str(link.attrs['href'])[2:] for link in objs]
# в итоге в links ссылки на каждую букву

# Получаем ссылки каждую группу (важно, что могут быть дубликаты)

In [ ]:
total_band_links=[]
for i,letter_page in enumerate(links):
    if(i%1000==0):
        print i
    try:
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    except:
        print i,'Сейчас исключение в request'е, переподключай интернет'
        time.sleep(30)
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    objs = soup.findAll('a')
    band_links=[str(link.attrs['href']) for link in objs]
    for j,band_link in enumerate(band_links):
        band_links[j]='https://www.azlyrics.com/'+band_link
    band_links=band_links[28:]
    band_links=band_links[:-7]
    total_band_links.append(band_links)

In [ ]:
flat_total_band_links = [item for sublist in total_band_links for item in sublist] #получаем 1d list

In [ ]:
np_total_band_links=np.array(flat_total_band_links)
unique_links=np.unique(np_total_band_links) #получаем уникальные

In [ ]:
np.savetxt("band_links.txt", unique_links,fmt='%s') #сохраняем ссылки на каждую группу.

# Теперь получим ссылки на каждую песню

In [3]:
unique_links=np.loadtxt('band_links.txt',dtype='str')

In [ ]:
total_song_links_list=[]
for i,page_link in enumerate(unique_links):
    if(i%1000==0):
        print i
    try:
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    except:
        print i,'Сейчас исключение в request'е, переподключай интернет'
        time.sleep(30)
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    obj = soup.find('div',attrs={'id':'listAlbum'})
    try:
        objs = obj.findAll('a')
    except:
        print page_link # здесь оказываются те, у кого нет альбомов, там другая разметка страницы
        continue
    help_list=[]
    for item in objs:
        if (not type(item.attrs.get('href'))==type(None)):
            help_list.append('https://www.azlyrics.com/'+str(item.attrs.get('href'))[3:])
    total_song_links_list.append(help_list)

In [ ]:
flat_total_song_links_list = [item for sublist in total_song_links_list for item in sublist] #получаем 1d list
np_song_links=np.array(flat_total_song_links_list)

In [6]:
np.savetxt("song_links.txt", np_song_links,fmt='%s') #сохраняем ссылки на каждую песню.

NameError: name 'np_song_links' is not defined

# Теперь непосредственно получаем текст песни, Название, Исполнителя, год и Альбом

In [17]:
df=pd.DataFrame(columns=['Artist','Song_name','Album','Year','Text'])

In [18]:
song_links=np.loadtxt('song_links.txt',dtype='str')

In [19]:
for i,page_link in enumerate(song_links):
    if(i%1000==0):
        print i
    try:
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    except:
        print i,'Сейчас исключение в requestе, переподключай интернет'
        time.sleep(30)
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    obj=soup.find('div',{'class':'col-xs-12 col-lg-8 text-center'})
    band_song = obj.findAll('b')
    band_name=str(band_song[0].text)[:-7]
    song_name=str(band_song[1].text)[1:-1]
    obj=soup.find('div',{'class':'col-xs-12 col-lg-8 text-center'})
    song_text=str(obj.findAll('div')[6].text).replace('\n' ,' ').replace('\42' ,'').replace('\r' ,'')
    alb=soup.find('a',{'data-toggle':'collapse'})
    if(type(alb)==type(None)):
        alb_name=np.nan
        year=np.nan
    else:
        alb_name=str(alb.text)[1:-8]
        year=str(alb.text)[-5:-1]
    data_row = {"Artist":band_name, "Song_name":song_name,"Album":alb_name, "Year":year,"Text":song_text}
    df = df.append(data_row, ignore_index=True)

0


In [20]:
df

,Artist,Song_name,Album,Year,Text
0,Mac Lethal,Mermaid Pornography,"Men Are From Mars, Porn Stars Are From Earth",2002,Land mines wheel chairs toilet snakes I'm ro...
1,"Hopsin, Roscoe Dash, Machine Gun Kelly, Future...",XXL Freshmen 2012 Cypher - Part 1,NaN,NaN,"[Hopsin:] Welcome to the Ill Mind of Hopsin,..."
2,XXXTENTACION,HOPE,?,2018,Yeah Rest in peace to all the kids that lost...
3,Pia Mia,Hold On We're Going Home,The Gift,2013,"[Verse 1:] I got my eyes on you, you're ever..."
4,The Yardbirds,For Your Love,For Your Love,1965,(For your love) (For your love) (For your lo...
